In [100]:
print(f"To check the SSL certificate expiration date, we will use the OpenSSL library. This package provides a high-level interface to the functions in the OpenSSL library.")

To check the SSL certificate expiration date, we will use the OpenSSL library. This package provides a high-level interface to the functions in the OpenSSL library.


In [101]:
#!pip install pyopenssl

In [ ]:
from datetime import datetime, date
import OpenSSL
import ssl
import os
import json
from dotenv import load_dotenv
import warnings
warnings.filterwarnings("ignore")

In [103]:
''' pip install python-dotenv'''
load_dotenv() # will search for .env file in local folder and load variables

True

In [104]:
source_es_host = "http://localhost:9211"
# source_es_host = os.getenv('ES_DEV_V8_HOST')

In [105]:
source_es_host = source_es_host.replace("http://","").replace("https://","")
source_es_hostname = str(source_es_host.split(':')[0])
source_es_port = str(source_es_host.split(':')[1])

In [106]:
print(source_es_hostname, source_es_port)

localhost 9211


In [107]:
response_dict = {}

In [108]:
def get_ssl_certs_hosts(source_es_hostname, source_es_port):
    ''' check if the ES cluster has any ssl certs and get the expired date '''
    try:
        cert=ssl.get_server_certificate((source_es_hostname, source_es_port))
        x509 = OpenSSL.crypto.load_certificate(OpenSSL.crypto.FILETYPE_PEM, cert)
        bytes=x509.get_notAfter()
        # print(bytes)
        timestamp = bytes.decode('utf-8')
        # print (datetime.strptime(timestamp, '%Y%m%d%H%M%S%z').date().isoformat())
        # print(datetime.strptime(timestamp, '%Y%m%d%H%M%S%z'))
        ssl_expire_date = datetime.strptime(timestamp, '%Y%m%d%H%M%S%z')
        ssl_expire_date = "{}-{}-{}".format(str(ssl_expire_date.year).zfill(2), str(ssl_expire_date.month).zfill(2), str(ssl_expire_date.day).zfill(2))
    
        response_dict.update({"ssl_certs_expire_date" : ssl_expire_date})
        response_dict.update({"ssl_certs_expire_yyyymmdd" : int(ssl_expire_date.replace("-",""))})
    except Exception as e:
        response_dict.update({"ssl_certs_expire_date" : 'no_ssl_certs'})
        response_dict.update({"ssl_certs_expire_yyyymmdd" : 0})
        print(e)
    finally:
        return response_dict

In [109]:
response_dict = get_ssl_certs_hosts(source_es_hostname, source_es_port)

[WinError 10061] No connection could be made because the target machine actively refused it


In [110]:
print(json.dumps(response_dict, indent=2)) 

{
  "ssl_certs_expire_date": "no_ssl_certs",
  "ssl_certs_expire_yyyymmdd": -1
}


In [ ]:
print(f"** The number of days remaining until the certificate expires from today's date.**")

In [ ]:
def subtract_dates_yyyymmdd(date_str1, date_str2):
    """
    Calculates the number of days between two dates in YYYYMMDD format.

    Args:
        date_str1 (str): The first date string in YYYYMMDD format.
        date_str2 (str): The second date string in YYYYMMDD format.

    Returns:
        int: The number of days between the two dates.
    """
    date1 = datetime.strptime(date_str1, "%Y-%m-%d")
    date2 = datetime.strptime(date_str2, "%Y-%m-%d")
    
    time_difference = date1 - date2
    return time_difference.days

In [ ]:
# Example usage:
date1_str = "2035-08-09"
# date2_str = "2023-01-15"
date2_str = date.today().strftime("%Y-%m-%d")
days_difference = subtract_dates_yyyymmdd(date1_str, date2_str)

In [ ]:
# threshold_days_of_expiration_date = 3650
threshold_days_of_expiration_date = 60

In [ ]:
if int(days_difference) < threshold_days_of_expiration_date:
    print(f"[Need to re-generation a ssl certificate] The difference between {date1_str} and {date2_str} is {days_difference} days.")
else:
    print(f"[Ok] The difference between {date1_str} and {date2_str} is {days_difference} days.")